In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold,StratifiedKFold
import pickle
from sklearn.ensemble import RandomForestClassifier as RFC
    



/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#lodaing data
data_path = "/home/raku/kaggleData/2sigma/xgb142/"
#store = "/home/raku/kaggleData/2sigma/rf2000/"
train_file = data_path + "xgb1.42-train.json"
test_file = data_path + "xgb1.42-test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)

feature_file = data_path+'xgb142features.pickle'
fileObject = open(feature_file,'r') 
features = pickle.load(fileObject)
fileObject.close()

(49352, 293)
(74659, 292)


In [4]:
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 2333)



In [5]:
cv_scores=[]
i=0


In [6]:
models=[]
for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    rf = RFC(100)
    rf.fit(dev_X,dev_y)
    preds = rf.predict_proba(val_X)

    loss = log_loss(val_y, preds)
    models.append(rf)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 0.600911631696
loss for the turn 2 is 0.580553400726
loss for the turn 3 is 0.584622910405
loss for the turn 4 is 0.590361613409
loss for the turn 5 is 0.607020731295
The mean of the cv_scores is:
0.592694057506


In [7]:
feature_importance=[]
importance_list= rf.feature_importances_
for i in range(len(features)):
    feature_importance.append((features[i],importance_list[i]))
feature_importance=sorted(feature_importance,key=lambda x:x[1],reverse=True)

In [8]:
pd_feature=pd.DataFrame(feature_importance)

In [9]:
pd_feature.columns=['feature','importance']

In [10]:
pd_feature['importance']=pd_feature['importance']/np.sum(pd_feature['importance'])

In [11]:
pd_feature['cumsum']=pd_feature['importance'].cumsum()

In [32]:
pd_feature.loc[pd_feature['cumsum']>0.99]

,feature,importance,cumsum
145,house_type_num_features_median,0.000635,0.990027
146,with_private_outdoor_space,0.000632,0.990659
147,bathrooms,0.000607,0.991266
148,house_type_bathrooms_median,0.000599,0.991865
149,house_type_bathrooms_min,0.000585,0.992450
150,with_common_outdoor_space,0.000585,0.993035
151,manager_id_bedrooms_min,0.000533,0.993568
152,with_wheelchair_access,0.000489,0.994057
153,with_fireplace,0.000414,0.994471
154,with_multi_level,0.000412,0.994883


In [12]:
feature95 = list(pd_feature.loc[pd_feature['cumsum']<0.95,'feature'])

In [13]:
feature99 = list(pd_feature.loc[pd_feature['cumsum']<0.99,'feature'])

In [14]:
feature_file = data_path+'xgb142features95.pickle'
fileObject = open(feature_file,'wb') 
pickle.dump(feature95,fileObject)
fileObject.close()

In [15]:
feature_file = data_path+'xgb142features99.pickle'
fileObject = open(feature_file,'wb') 
pickle.dump(feature99,fileObject)
fileObject.close()

In [40]:
models=[]
i=0
for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[feature95].as_matrix(), val_set[feature95].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    rf = RFC(100)
    rf.fit(dev_X,dev_y)
    preds = rf.predict_proba(val_X)

    loss = log_loss(val_y, preds)
    models.append(rf)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 0.599542188649
loss for the turn 2 is 0.588280494167
loss for the turn 3 is 0.583473176134
loss for the turn 4 is 0.59353558567
loss for the turn 5 is 0.59288112862
The mean of the cv_scores is:
0.590615942305


In [41]:
models=[]
i=0
for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[feature99].as_matrix(), val_set[feature99].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    rf = RFC(100)
    rf.fit(dev_X,dev_y)
    preds = rf.predict_proba(val_X)

    loss = log_loss(val_y, preds)
    models.append(rf)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 0.594212203085
loss for the turn 2 is 0.592228522858
loss for the turn 3 is 0.577352429753
loss for the turn 4 is 0.594768213229
loss for the turn 5 is 0.595310629226
The mean of the cv_scores is:
0.590665460219
